In [1]:
# All imports
import numpy as np
import matplotlib.pyplot as plt

In [79]:
start_price = 131
# percentage volatility or annualized volatility
vola = 0.25
# Overall trend or average return per year
drift = 0.35 
# Total number of simulations
n_simul = 1000
# Assumption 252 trading days
delta = 1.0 / 252.0
# Simulation years
sim_years = 5
# Stockdays per year
trad_days_per_year = 252
# Runtime in days
runtime = trad_days_per_year * sim_years - 1

In [64]:
def brownian_motion(delta, vola, runtime, n_simul):
    # standard deviation or scale is sqrt of volatiliy for brownian motion
    return vola * np.random.normal(loc=0, scale=np.sqrt(delta),
                                   size=(runtime, n_simul))

In [65]:
# Random geometric motion returns
def rgm_returns(delta, vola, runtime, drift, n_simul):
    motion = brownian_motion(delta, vola, runtime, n_simul)
    return np.exp(motion + (drift - vola ** 2 / 2) * delta)

In [66]:
def rgm_stock_movements(start_price, delta, vola, 
                        runtime, drift, n_simul):
    returns = rgm_returns(delta, vola, runtime, drift, n_simul)
    stacked = np.vstack([np.ones(n_simul), returns]) # prepends 1 infront of all returns
    return start_price * stacked.cumprod(axis=0)

In [ ]:
stock_prices = rgm_stock_movements(start_price, delta, 
                                   vola, runtime, drift, 
                                   n_simul)

In [ ]:
plt.plot(stock_prices, linewidth=0.25)
plt.show()

In [69]:
# This code shows one line on the stock projections
example_stock_flow = stock_prices[:, 0]

In [70]:
# Withdrawal rate is yearly and after the first year!
yearly_withdrawels = True
withdraw_after_year = True


# Money is taken out of the account after one year
withdrawel_rate = 0.04

# Create an index array, that is used for later withdrawals
example_stock_flow_index = np.arange(1, runtime+2)

# show an example withdrawel array for example_stock_flow
example_withdrawals = np.where(example_stock_flow_index % 252 == 0, example_stock_flow * withdrawel_rate, 0)

In [ ]:
# control print statements
print(example_stock_flow_index.shape)
print(example_stock_flow.shape)
print(example_withdrawals[251])
print(example_withdrawals[503])
print(example_withdrawals[755])
print(example_withdrawals[1007])
print(example_withdrawals[1259])

In [ ]:
total_withdrawals = np.sum(example_withdrawals)
avg_withdrawals = total_withdrawals / sim_years

print(total_withdrawals)
print(avg_withdrawals)

In [ ]:
example_stock_flow_index

In [ ]:
example_withdrawals

In [76]:
def w_returns(withdrawel_rate, runtime):
    # Creates an index array to know when to withdraw
    index_arr = np.arange(1, runtime+2)

    # show an example withdrawel array for example_stock_flow
    returns = np.where(index_arr % trad_days_per_year == 0, -withdrawel_rate, 0)
    return returns

In [ ]:
example_withdrawals = w_returns(withdrawel_rate, runtime)
example_withdrawals = np.expand_dims(example_withdrawals, axis=1)
example_withdrawals = np.repeat(example_withdrawals, axis=1, repeats=1000)
example_withdrawals[:, 80]

In [111]:
def rgm_stock_mov_plus_withdrawal(start_price, delta, vola, 
                                  runtime, drift, n_simul,
                                  withdrawel_rate):
    market_returns = rgm_returns(delta, vola, runtime, drift, n_simul)
    withdrawal_returns = w_returns(withdrawel_rate, runtime)
    
    # prepends 1 infront of all market returns
    market_returns_stacked = np.vstack([np.ones(n_simul), market_returns])
    
    withdrawal_returns = np.expand_dims(withdrawal_returns, axis=1)
    withdrawal_returns = np.repeat(withdrawal_returns, axis=1, repeats=n_simul)

    tot_returns = market_returns_stacked + withdrawal_returns
    return start_price * tot_returns.cumprod(axis=0)

In [126]:
stock_prices = rgm_stock_mov_plus_withdrawal(start_price, delta, 
                                            vola, runtime, drift, 
                                            n_simul, withdrawel_rate=0.16)

In [ ]:
plt.plot(stock_prices, linewidth=0.25)
plt.show()

In [ ]:
# Notes David:
# Next step, when zero, make total return equal to 0 for period after!
# Calculate total withdrawals made during the time!
# Start working on dinamic plot